In [1]:
import gym
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
import tensorflow as tf

%load_ext tensorboard
def playGame(env):
    env.reset()
    done = False
    sumreward = 0
    observ = []
    actions = []
    rewards = []
    while not done:
        action = env.action_space.shape
        obs,reward,done, info = env.step(action)
        actions.append([action])
        observ.append(obs)
        rewards.append(reward)
        sumreward += reward
        
    observ  = np.asarray(observ)
    actions = to_categorical(np.asarray(actions)) * np.asarray(rewards).reshape(-1,1)
    return observ,actions
    

In [29]:
env = gym.make("CartPole-v1")
INPUT_SHAPE = env.observation_space.shape
print(INPUT_SHAPE)
NUM_ACTIONS = env.action_space.n

(4,)


In [25]:

def createRamModel(): 
    
    model = tf.keras.Sequential()
    model.add(layers.Dense(60, activation='relu',input_shape=INPUT_SHAPE))
    model.add(layers.Dense(40,activation='relu'))
    # output is the same size as number of outputs
    model.add(layers.Dense(NUM_ACTIONS))
    model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error
    return model

memory_actions = []
training_model = createRamModel()
target_model = createRamModel()

In [27]:
def run():
    firstobs = env.reset()
    print(target_model.predict(firstobs))

In [28]:
run()

ValueError: in user code:

    /home/alice/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    /home/alice/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/alice/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/alice/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/alice/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1122 predict_step  **
        return self(x, training=False)
    /home/alice/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:885 __call__
   

In [ ]:
import time
obs = env.reset().reshape(1,-1)
done = False
action = np.argmax(model.predict(obs)[0])

while not done:
    env.render()
    time.sleep(1)
    obs,reward,done, info = env.step(action)
    obs = np.asarray(obs).reshape(1,-1)
    action = np.argmax(model.predict(obs)[0])
